## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

_IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts._

In [1]:
# Install Strands Agents and other necessary libraries
%pip install -q --force-reinstall strands-agents strands-agents-tools "botocore>=1.40.26" "boto3>=1.40.26" "awscli>=1.29.57"

Note: you may need to restart the kernel to use updated packages.


In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Import libraries and initialize boto3 session and config

In [3]:
import boto3
import json
import base64
from datetime import datetime
from botocore.config import Config
from strands import Agent
from strands.models import BedrockModel
from pprint import pprint

LITE_MODEL_ID = "us.amazon.nova-2-lite-v1:0"

# Create a Bedrock config
boto_config = Config(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)
boto_session = boto3.Session(
    region_name='us-east-1',
)



### Initialize Strands Agents powered by Nova 

In [4]:
# Bedrock
bedrock_model = BedrockModel(
  model_id=LITE_MODEL_ID,
  boto_session=boto_session,
  boto_client_config=boto_config,
  temperature=0.7,
  streaming=True, # default is True, Enable/disable streaming
  max_tokens=2048 # Adjust it for your needs
)
agent = Agent(model=bedrock_model)

agent("How many times character R occurs in the world 'strawberry' ")


To determine how many times the character **R** occurs in the word **"strawberry"**, let's follow these steps:

---

### Step 1: Write out the word  
The word is:  
**s t r a w b e r r y**

---

### Step 2: Look at each character  
Let's list each letter in order:

1. **s**
2. **t**
3. **r**
4. **a**
5. **w**
6. **b**
7. **e**
8. **r**
9. **r**
10. **y**

---

### Step 3: Count the occurrences of **R**

Now, go through the list and count every time **r** (or **R**, though case doesn't matter here) appears:

- Position 3: **r** → count = 1  
- Position 8: **r** → count = 2  
- Position 9: **r** → count = 3  

---

### ✅ Final Answer:  
$$
\boxed{3}
$$

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'To determine how many times the character **R** occurs in the word **"strawberry"**, let\'s follow these steps:\n\n---\n\n### Step 1: Write out the word  \nThe word is:  \n**s t r a w b e r r y**\n\n---\n\n### Step 2: Look at each character  \nLet\'s list each letter in order:\n\n1. **s**\n2. **t**\n3. **r**\n4. **a**\n5. **w**\n6. **b**\n7. **e**\n8. **r**\n9. **r**\n10. **y**\n\n---\n\n### Step 3: Count the occurrences of **R**\n\nNow, go through the list and count every time **r** (or **R**, though case doesn\'t matter here) appears:\n\n- Position 3: **r** → count = 1  \n- Position 8: **r** → count = 2  \n- Position 9: **r** → count = 3  \n\n---\n\n### ✅ Final Answer:  \n$$\n\\boxed{3}\n$$'}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[1.6488769054412842], traces=[<strands.telemetry.metrics.Trace object at 0x1067c1d80>], accumulated_usage={'inputTokens': 59, 'outpu

### Structured Output with Agent powered by Nova

In [5]:
from pydantic import BaseModel, Field
from typing import List, Optional

# 1) Define the Pydantic model
class PersonInfo(BaseModel):
    """Model that contains information about a Person"""
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    occupation: str = Field(description="Occupation of the person")

# 2) Pass the model to the agent
agent = Agent(model=bedrock_model)
result = agent(
    "John Smith is a 30 year-old software engineer",
    structured_output_model=PersonInfo
)

# 3) Access the `structured_output` from the result
person_info: PersonInfo = result.structured_output
pprint(f"Name: {person_info.name}")      # "John Smith"
pprint(f"Age: {person_info.age}")        # 30
pprint(f"Job: {person_info.occupation}") # "software engineer"


Tool #1: PersonInfo
'Name: John Smith'
'Age: 30'
'Job: software engineer'


### Streaming Structured output with Agent powered by Nova

In [6]:
from pydantic import BaseModel, Field
from typing import List, Optional

# 1) Define the Pydantic model
class PersonInfo(BaseModel):
    """Model that contains information about a Person"""
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    occupation: str = Field(description="Occupation of the person")

# 2) Pass the model to the agent
streaming_agent = Agent(model=bedrock_model)
async for event in streaming_agent.stream_async(
    "John Smith is a 30 year-old software engineer",
    structured_output_model=PersonInfo
):
    if "data" in event:
        pprint(event["data"], end="", flush=True)
    elif "result" in event:
        print(f'Results is:\n {event["result"].structured_output}')



Tool #1: PersonInfo
Results is:
 name='John Smith' age=30 occupation='software engineer'


### Combine Strand tool - calculator with Agent powered by Nova

In [7]:
from pydantic import BaseModel, Field
from strands_tools import calculator


class MathResult(BaseModel):
    operation: str = Field(description="the performed operation")
    result: int = Field(description="the result of the operation")

tool_agent = Agent(model=bedrock_model, tools=[calculator]
)
res = tool_agent("What is 2^64", structured_output_model=MathResult)
pprint(res.structured_output)


Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬──────────────────────╮                                                                           │
│  │ Operation │ Evaluate Expression  │                                                                           │
│  │ Input     │ 2**64                │                                                                           │
│  │ Result    │ 18446744073709551616 │                                                                           │
│  ╰───────────┴──────────────────────╯                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Tool #2: MathResult
MathResult(operation='2^64', result=18446744073709551616)
